In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import requests

ticker = 'meli'
ticker = ticker.upper()
today = dt.date.today()

In [ ]:
def get_ema(ticker, today = today):
    ema_values = []
    for short_ema in range(2, 51):
        for long_ema in range(10, 91):
            if short_ema < long_ema:
                data = yf.download(tickers = ticker, start = '2010-1-1', end = f'{today.year}-{today.month}-{today.day}', auto_adjust = True)
                data[f'EMA_{short_ema}'] = data['Close'].ewm(span = short_ema).mean()
                data[f'EMA_{long_ema}'] = data['Close'].ewm(span = long_ema).mean()
                data['Ratio'] = (data[f'EMA_{short_ema}'] / data[f'EMA_{long_ema}']).shift()
                data['State'] = data.apply(lambda row: 'In' if row['Ratio'] >= 1 else 'Out', axis = 1)
                data['Yield'] = data['Close'].pct_change() + 1

                in_asset = data[data['State'] == 'In']
                cumulative_yield = (in_asset['Yield'].prod() - 1)
                start_date = in_asset.index[0].date()
                days = (today - start_date).days
                annual_yield = round((((1 + cumulative_yield) ** (365 / days)) - 1) * 100, 2)
                ema_values.append([short_ema, long_ema, annual_yield])

    return ema_values

ema_values = get_ema(ticker = ticker)
print(ema_values)

In [ ]:
def create_ema_df(data):
    ema_columns = ['short_ema', 'long_ema', 'yield']
    ema_df = pd.DataFrame(data, columns = ema_columns)
    ema_df.sort_values(by = 'yield', ascending = False, inplace = True)
    ema_df = ema_df.iloc[:10, :]
    
    return ema_df

ema_df = create_ema_df(data = ema_values)
ema_df

In [ ]:
def mean_emas(data, ticker, today = today):
    mean_short_ema = int(data['short_ema'].mean())
    mean_long_ema = int(data['long_ema'].mean())
    mean_yield = int(data['yield'].mean())
    data = yf.download(tickers = ticker, start = '2010-1-1', end = f'{today.year}-{today.month}-{today.day}', auto_adjust = True)
    data[f'EMA_{mean_short_ema}'] = data['Close'].ewm(span = mean_short_ema).mean()
    data[f'EMA_{mean_long_ema}'] = data['Close'].ewm(span = mean_long_ema).mean()
    data['Ratio'] = (data[f'EMA_{mean_short_ema}'] / data[f'EMA_{mean_long_ema}']).shift()
    data['State'] = data.apply(lambda row: 'In' if row['Ratio'] >= 1 else 'Out', axis = 1)
    data['Action'] = ['Buy' if (data['State'][i] == 'In') and (data['State'][i - 1] == 'Out') else 'Sell' if (data['State'][i] == 'Out') and (data['State'][i - 1] == 'In') else 'Pass' for i in range(len(data))]

    sell_signals = data[data['Action'] == 'Sell']
    buy_signals = data[data['Action'] == 'Buy']
    
    plt.style.use('dark_background')
    fig, ax = plt.subplots(figsize = (40, 16))
    ax.scatter(x = sell_signals.index, y = sell_signals['Close'], color = 'red', marker = 'v', label = 'Sell', s = 200)
    ax.scatter(x = buy_signals.index, y = buy_signals['Close'], color = 'green', marker = '^', label = 'Buy', s = 200)
    sns.lineplot(data = data, x = data.index, y = data['Close'], label = 'Stock Price')
    sns.lineplot(data = data, x = data.index, y = data[f'EMA_{mean_short_ema}'], label = f'Short EMA ({mean_short_ema})')
    sns.lineplot(data = data, x = data.index, y = data[f'EMA_{mean_long_ema}'], label = f'Long EMA ({mean_long_ema})')
    plt.legend(fontsize = 'large')
    plt.title(f'Rendimiento acumulado anual de {ticker}: {mean_yield}%')
    plt.show()

    return data

mean_emas_data = mean_emas(data = ema_df, ticker = ticker)

In [ ]:
def sp_companies():
    url = 'https://www.slickcharts.com/sp500'
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url = url, headers = headers)
    sp_data = pd.read_html(r.text)[0][['Company', 'Symbol']]

    return sp_data

sp_companies()

In [ ]:
def stock_correlations(ticker = ticker, sp_companies = sp_companies(), today = today):
    tickers = sp_companies['Symbol'].to_list()
    tickers = [s.replace('.', '-') for s in tickers]
    if ticker in tickers:
        tickers
    else:
        tickers.append(ticker)

    data = yf.download(tickers = tickers, start = '2010-1-1', end = f'{today.year}-{today.month}-{today.day}', auto_adjust = True)['Close']
    data = data.pct_change()
    correlation = data.corr()[ticker]
    correlation = correlation.sort_values(ascending = False)
    correlation = correlation.iloc[1:11]
    series_df = correlation.reset_index()
    series_df.columns = ['Symbol', 'Correlation']
    correlation = pd.merge(left = series_df, right = sp_companies, how = 'left', left_on= 'Symbol', right_on = 'Symbol')
    correlation.rename(columns = {'Company': 'Name'}, inplace = True)
    correlation.set_index('Symbol', inplace = True)
    
    return correlation

best_stocks = stock_correlations()
best_stocks

In [ ]:
def commodities():
    url = 'https://finance.yahoo.com/commodities/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAACxVMXbUY4TFerSQ8G61bHBLfBhaXr7kfzXI_kTs9WfeaaeqQ_x_RHqaiaf9mZxlSxo28LHouGXWcFWxWnhDnotI6yS50Efs0VyFDt_dHPHraJ4h9S9gkI1t31lN2tTfNPGrYPmmSjv4ekDEoqk72McrbL7QSs1mYtRaKsX7SB4A'
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url = url, headers = headers)
    commodities = pd.read_html(r.text)[0]
    commodities = commodities.loc[:, ['Symbol', 'Name']]

    return commodities

commodities()

In [ ]:
def commodities_correlations(ticker = ticker, commodities = commodities(), today = today):
    tickers = commodities['Symbol'].to_list()
    tickers.append(ticker)
    data = yf.download(tickers = tickers, start = '2010-1-1', end = f'{today.year}-{today.month}-{today.day}', auto_adjust = True)['Close']
    data = data.pct_change()
    correlation = data.corr()
    correlation = correlation[ticker]
    correlation = correlation.sort_values(ascending = False)
    correlation = correlation.iloc[1:4]
    df_correlation = correlation.reset_index()
    df_correlation.columns = ['Symbol', 'Correlation']
    correlation = pd.merge(left = df_correlation, right = commodities, how = 'left', left_on = 'Symbol', right_on = 'Symbol')
    correlation = correlation.loc[:, ['Symbol', 'Correlation', 'Name']]
    correlation.set_index('Symbol', inplace = True)

    return correlation

best_commodities = commodities_correlations()
best_commodities

In [ ]:
def best_correlations(stocks = best_stocks, commodities = best_commodities):
    data = pd.concat(objs = [stocks, commodities])
    data.sort_values(by = 'Correlation', ascending = False, inplace = True)

    return data

best_correlations()

In [ ]:
def graphic_correlations(data = best_correlations(), ticker = ticker, today = today):
    tickers = data.index.to_list()
    tickers.append(ticker)
    values = yf.download(tickers = tickers, start = '2010-1-1', end = f'{today.year}-{today.month}-{today.day}', auto_adjust = True)['Close']
    values.dropna(inplace = True)
    values = values.divide(values.iloc[0]) * 100

    plt.style.use('dark_background')
    fig, ax = plt.subplots(figsize = (40, 25))
    for ticker in values:
        plt.plot(values.index, values[ticker], label = f'{ticker}')
        plt.annotate(ticker, xy=(values.index[-1], values[ticker].iloc[-1]), xytext=(5,0), textcoords='offset points', ha='left', va='center', color='white', size = 15)
    plt.legend(fontsize = 'x-large')
    return values

graphic_correlations()